In [229]:
import numpy as np
import numpy.lib.mixins
from numbers import Number
from math import *

def conv(n, N=52):
    n = int(n) if isinstance(n, str) else n
    n = math.ceil(n) if isinstance(n, float) else n
    n = n % N    
    return N - (n-1)

def iconv(n, N=52):
    n = n % N
    return (N - n) + 1

def sconv(n, N=52):
    return str(iconv(n, N))

HANDLED_FUNCTIONS = {}

def implements(np_function):
   "Register an __array_function__ implementation for DiagonalArray objects."
   def decorator(func):
       HANDLED_FUNCTIONS[np_function] = func
       return func
   return decorator

@implements(np.sum)
def sum(arr):
    "Implementation of np.sum for DiagonalArray objects"
    return arr._i * arr._N

@implements(np.mean)
def mean(arr):
    "Implementation of np.mean for DiagonalArray objects"
    return arr._i / arr._N

def binfmt(N=52):
    return str('{:0>'+str(N)+'s}').format

def arr2str(arr):
    return ''.join(np.array(arr).astype(str))

def align_bin(val, N=52):
    
    return binfmt(N)(str(val))

def bin2intarr(v):
    return np.array(list(v[2:] if str(v).startswith('0b') else v)).astype(int) 

def nfill(arr, N=52, val=0):
    s = [val]*(N-len(arr))
    return np.concatenate([s, arr])

def binarr_to_values(val, N=52):
    val = nfill(val,N,0)#bin2intarr(align_bin(arr2str(val), N))
    return np.nonzero(np.where(val>0, pool(N), 0))

def pool(N=52):
    return N - np.arange(N) 

def nmax(N=52):
    return int(str('{:0<'+str(N)+'s}').format('111111'), 2)  

class ResultsArray(numpy.lib.mixins.NDArrayOperatorsMixin):

    def __init__(self, N=52, value=(1,2,)):
        self._N = pool(N)
        if len(value) == N:
            if all(np.isin(np.array(value), [0,1])):
                value = binarr_to_values(value, N)
        self._i = value #[conv(n, N) for n in value]
        self._m = nmax(N)  

    @property
    def max_val(self):
        return self._m

    def __repr__(self):
        return f"{self.__class__.__name__}(N={len(self._N)}, value={self._i})"
    
    def __str__(self):
        return str(self._i)
         
    def __int__(self):
        return int(''.join(np.array(self).astype(str)), 2)

    def __float__(self):
        return (int(self) * 1.0) / self.max_val
    
    @classmethod
    def from_array(cls, val, N=52):
        return cls(N=N, value=binarr_to_values(val, N))
    
    @classmethod
    def from_float(cls, f, N=52):
        mx = int(''.join(np.where(np.arange(N) < 6, 1, 0).astype(str)), 2) * 1.0
        return cls.from_int(int(round(f * mx)), N)
    
    @classmethod
    def from_int(cls, val, N=52):
        return cls(N=N, value=[N - (i + 1) for i, n in enumerate(list(bin(val)[2:])) if n == '1'])
    
    def __array__(self):
        return np.where(np.isin(self._N, self._i), 1, 0) 

    def __array_ufunc__(self, ufunc, method, *inputs, **kwargs):
        if method == '__sub__':
            return self.__class__.from_int(int(self) & int(inputs[0]))
        if method == '__call__':
            N = None
            scalars = []
            for input in inputs:
                # In this case we accept only scalar numbers or ResultsArray.
                if isinstance(input, Number):
                    scalars.append(input)
                elif isinstance(input, self.__class__):
                    scalars.append(np.array(input))
                    N = len(self._N)                    
                else:
                    return NotImplemented
            return self.__class__(N, binarr_to_values(ufunc(*scalars, **kwargs)))
        else:
            return NotImplemented
    
#     def __array_function__(self, func, types, args, kwargs):
#        if func not in HANDLED_FUNCTIONS:
#            return NotImplemented
#        # Note: this allows subclasses that don't override
#        # __array_function__ to handle ResultsArray objects.
#        if not all(issubclass(t, self.__class__) for t in types):
#            return NotImplemented
#        return HANDLED_FUNCTIONS[func](*args, **kwargs)

def rarray(v, N=52):
    return ResultsArray(N=N, value=v)


In [230]:
a = rarray([3, 41, 37, 28, 51, 42])

In [231]:
a

ResultsArray(N=52, value=[3, 41, 37, 28, 51, 42])

In [232]:
fa = float(a)

In [233]:
str(a)

'[3, 41, 37, 28, 51, 42]'

In [234]:
nz = np.array([0]*52)
ndn = np.array([1,0,0,0,1,1,1,0,0,0,1])


In [235]:
s = ResultsArray.from_float(fa)
str(s)

'[51, 42, 41, 37, 28, 3]'

In [237]:
v = ResultsArray(value=[5, 40, 15, 10, 42, 51])
dif = v-a

In [127]:
nv = 1-np.array(v)
na = np.array(a)
nnd = np.abs(nv - na)
pool = 52 - np.arange(52)
nnd


array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0,
       1, 1, 1, 1, 0, 1, 0, 1])

In [242]:
np.dot(np.array(a), np.array(dif).T)

2

In [116]:
diff = ResultsArray.from_float(float(v) - float(a))

In [117]:
print(bin(int(v)) )

ResultsArray(N=52, value=[3, 6, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 29, 30, 31, 32, 34, 35, 36, 37, 38, 40])

In [118]:
int(v) - int(a)

-309304745722

In [119]:
1-na

array([1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 1])